# Alternative PygmalionAI notebook for Google Colab

This notebook is an *unofficial* method to run the [PygmalionAI LLMs](https://huggingface.co/PygmalionAI) (NSFW) using **oobabooga**'s [text-generation-webui](https://github.com/oobabooga/text-generation-webui) project in chat mode.

Run all the cells and a public gradio URL will appear at the bottom in around 12 minutes.

You can create your own custom characters using the [Pygmalion JSON character creator](https://oobabooga.github.io/character-creator.html)

Original author: https://github.com/81300

In [ ]:
#@title 1. Keep this tab alive to prevent Colab from disconnecting you { display-mode: "form" }

# From: https://github.com/henk717/KoboldAI

%%html
<b>Press play on the music player to keep the tab alive (Uses only 13MB of data)</b><br/>
<audio src="https://henk.tech/colabkobold/silence.m4a" controls>

In [ ]:
#@title 2. Select the model and start the web UI

%cd /tmp
!wget -c https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
%env PYTHONPATH=
!which conda 2>/dev/null || bash Miniconda3-latest-Linux-x86_64.sh -b -p /usr/local -f
%env CI=true
!conda init -q bash
!source /usr/local/bin/activate && conda update -y -q -c defaults --all
!source /usr/local/bin/activate && conda create -y -q -n textgen
!source /usr/local/bin/activate && conda activate textgen && conda install -y -q -c defaults conda python
!source /usr/local/bin/activate && conda activate textgen && conda install -y -q -c pytorch -c nvidia pytorch torchvision torchaudio pytorch-cuda=11.7
!source /usr/local/bin/activate && conda clean -y -q --all

# Download Github repo
%cd /content
!git clone https://github.com/oobabooga/text-generation-webui
!ln -s text-generation-webui/models .
!ln -s text-generation-webui/characters .
!ln -s text-generation-webui/presets .
%cd text-generation-webui
!wget https://oobabooga.github.io/settings-colab.json

# Install requirements
!source /usr/local/bin/activate && conda activate textgen \
  && pip install --no-cache-dir --progress-bar=off --root-user-action=ignore -r requirements.txt diffusers

Model = "Pygmalion 6B-original (yields better results)" #@param ["Pygmalion 6B-original (yields better results)", "Pygmalion 6B", "Pygmalion 6B experimental", "Pygmalion 2.7B", "Pygmalion 1.3B", "Pygmalion 350M"] {allow-input: false}

if Model == "Pygmalion 6B-original (yields better results)":
  huggingface_org = "PygmalionAI"
  huggingface_repo = "pygmalion-6b"
  huggingface_branch = "b8344bb4eb76a437797ad3b19420a13922aaabe1"
  model_name = "pygmalion-6b_b8344bb4eb76a437797ad3b19420a13922aaabe1"
elif Model == "Pygmalion 6B":
  huggingface_org = "PygmalionAI"
  huggingface_repo = "pygmalion-6b"
  huggingface_branch = "main"
  model_name = "pygmalion-6b"
elif Model == "Pygmalion 6B experimental":
  huggingface_org = "PygmalionAI"
  huggingface_repo = "pygmalion-6b"
  huggingface_branch = "dev"
  model_name = "pygmalion-6b_dev"
elif Model == "Pygmalion 2.7B":
  huggingface_org = "PygmalionAI"
  huggingface_repo = "pygmalion-2.7b"
  huggingface_branch = "main"
  model_name = "pygmalion-2.7b"
elif Model == "Pygmalion 1.3B":
  huggingface_org = "PygmalionAI"
  huggingface_repo = "pygmalion-1.3b"
  huggingface_branch = "main"
  model_name = "pygmalion-1.3b"
elif Model == "Pygmalion 350M":
  huggingface_org = "PygmalionAI"
  huggingface_repo = "pygmalion-350m"
  huggingface_branch = "main"
  model_name = "pygmalion-350m"

# Download the model
![[ ! -f models/$model_name/config.json ]] && source /usr/local/bin/activate && conda activate textgen \
  && python download-model.py $huggingface_org/$huggingface_repo --branch $huggingface_branch

!source /usr/local/bin/activate && conda activate textgen \
  && python server.py --share --cai-chat --no-stream --auto-devices --model $model_name --settings settings-colab.json